In [1]:
library(ggplot2)
library(lattice)
library(rpart)
library(rattle)
library(caret)
adult = read.csv("C:\\Users\\student\\Documents\\Statistical Analysis using R\\Assignment 10\\adult.csv", header = FALSE)
colnames(adult) <- c("age", "workclass", "fnlwgt", "education", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "capital_gain", "capital_loss", "hours_per_week", "country", "salary_bucket")

Rattle: A free graphical interface for data science with R.
Version 5.2.0 Copyright (c) 2006-2018 Togaware Pty Ltd.
Type 'rattle()' to shake, rattle, and roll your data.


In [2]:
levels(adult$workclass)[levels(adult$workingclass)==" ?"]<-NA
levels(adult$occupation)[levels(adult$occupation)==" ?"]<-NA
levels(adult$country)[levels(adult$country)==" ?"]<-NA

In [3]:
replaceMissingValues = function(dataSet) {
  for(j in 1:ncol(dataSet)) {
    if(is.factor(dataSet[,j])) {
      dataSet[,j][is.na(dataSet[,j])]<-names(sort(table(dataSet[,j])))[1]
    }
  }
  return(dataSet)
}
adult=replaceMissingValues(adult)

In [4]:
library(caret)
preProcessDataModel <- preProcess(adult, method='knnImpute')
preProcessDataModel

Created from 32561 samples and 15 variables

Pre-processing:
  - centered (6)
  - ignored (9)
  - 5 nearest neighbor imputation (6)
  - scaled (6)


In [5]:
levels(adult$country)<-c("Asia","North_America","Asia","South_America","North_America","North_America","South_America","North_America","Europe","Europe","Europe","Europe","North_America","North_America","Europe","North_America","Asia","Europe","Asia","Asia","Europe","Europe","North_America","Asia","Asia","North_America","South_America","Oceanic","South_America","Oceanic","Europe","Europe","North_America","Europe","Asia","Asia","Asia","North_America","South_America","Asia","Europe")
levels(adult$education)<-c("High_school","High_school","High_school","Elementary","Elementary","Elementary","High_school","Undergrad","Undergrad","Undergrad","Postgrad","High_school","Professiona","Elementary","Postgrad","Undergrad")
levels(adult$occupation)<-c("Business","Government","Industry","Business","Agriculture","Business","Industry","Other","Business","Business","Government","Business","Business","Industry")

In [6]:
if(!is.factor(adult$salary_bucket)) {
  adult$salary_bucket = factor(adult$salary_bucket)
}
dummies_model <- dummyVars(salary_bucket~., data=adult)


# Create the dummy variables using predict. The Y variable will not be present in trainData_mat.
trainData_mat <- predict(dummies_model, newdata = adult)

# # Convert to dataframe
trainData <- data.frame(trainData_mat)
trainData$salary_bucket <- adult$salary_bucket

adult = trainData

Warning message in model.frame.default(Terms, newdata, na.action = na.action, xlev = object$lvls):
"variable 'salary_bucket' is not a factor"

In [7]:
preProcess_missingdata_model <- preProcess(adult, method= c("center", "scale"))
preProcess_missingdata_model

Created from 32561 samples and 51 variables

Pre-processing:
  - centered (50)
  - ignored (1)
  - scaled (50)


In [8]:
set.seed(2018)
names(adult)[51]="salary_bucket"
splitIndex <- createDataPartition(adult$salary_bucket, p = 0.70, list = FALSE, times = 1)
trainData <- adult[splitIndex,]
testData <- adult[-splitIndex,]
mytree <- rpart(salary_bucket ~ ., data = trainData, method = "class")
pred <- predict(mytree, testData, type = "class")
cm=confusionMatrix(data = pred, reference = testData$salary_bucket, positive = " >50K")
dataAccuracy<-cm[[3]][1]
dataBalancedAccuracy<-cm[[4]][11]
dataAccuracy
dataBalancedAccuracy

Accuracy 
0.8443898

Balanced Accuracy 
        0.7407426

In [9]:
library(ranger)
modelRanForest = ranger(salary_bucket~.,data=trainData, classification = TRUE)
pred = predict(modelRanForest, data = testData)$predictions
cmf=confusionMatrix(as.factor(pred), as.factor(testData$salary_bucket), positive = " >50K")
cmf[[3]][1]
cmf[[4]][11]


Attaching package: 'ranger'

The following object is masked from 'package:rattle':

    importance



Accuracy 
0.8607699

Balanced Accuracy 
        0.7750465

In [10]:
replaceMissingValues = function(dataSet) {
  for(j in 1:ncol(dataSet)) {
    if(!is.numeric(dataSet[,j])) {
      levels(dataSet[,j])[levels(dataSet[,j])==" ?"]<-names(sort(table(dataSet[,j])))[1]
      levels(dataSet[,j])[levels(dataSet[,j])==" ."]<-names(sort(table(dataSet[,j])))[1]
      levels(dataSet[,j])[levels(dataSet[,j])==" NA"]<-names(sort(table(dataSet[,j])))[1]
    } else {
      dataSet[,j][is.na(dataSet[,j])]<-mean(dataSet[,j],na.rm=TRUE)
    }
  }
  return(dataSet)
}
adult2 = read.csv("C:\\Users\\student\\Documents\\Statistical Analysis using R\\Assignment 10\\adult.csv", header = FALSE)
colnames(adult2) <- c("age", "workclass", "fnlwgt", "education", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "capital_gain", "capital_loss", "hours_per_week", "country", "salary_bucket")
levels(adult2$workclass)[levels(adult2$workingclass)==" ?"]<-NA
levels(adult2$occupation)[levels(adult2$occupation)==" ?"]<-NA
levels(adult2$country)[levels(adult2$country)==" ?"]<-NA

levels(adult2$country)<-c("Asia","North_America","Asia","South_America","North_America","North_America","South_America","North_America","Europe","Europe","Europe","Europe","North_America","North_America","Europe","North_America","Asia","Europe","Asia","Asia","Europe","Europe","North_America","Asia","Asia","North_America","South_America","Oceanic","South_America","Oceanic","Europe","Europe","North_America","Europe","Asia","Asia","Asia","North_America","South_America","Asia","Europe")
levels(adult2$education)<-c("High_school","High_school","High_school","Elementary","Elementary","Elementary","High_school","Undergrad","Undergrad","Undergrad","Postgrad","High_school","Professiona","Elementary","Postgrad","Undergrad")
levels(adult2$occupation)<-c("Business","Government","Industry","Business","Agriculture","Business","Industry","Other","Business","Business","Government","Business","Business","Industry")
adult2<-replaceMissingValues(adult2)

set.seed(2018)
names(adult2)[15]="salary_bucket"
splitIndex <- createDataPartition(adult$salary_bucket, p = 0.70, list = FALSE, times = 1)
trainData <- adult2[splitIndex,]
testData <- adult2[-splitIndex,]
mytree <- rpart(salary_bucket ~ ., data = trainData, method = "class")
pred <- predict(mytree, testData, type = "class")
cm=confusionMatrix(data = pred, reference = testData$salary_bucket, positive = " >50K")
dataAccuracy<-cm[[3]][1]
dataBalancedAccuracy<-cm[[4]][11]
dataAccuracy
dataBalancedAccuracy

Accuracy 
0.8456183

Balanced Accuracy 
        0.7414065

In [11]:
preProcessDataModel <- preProcess(adult, method='knnImpute')
preProcessDataModel

set.seed(2018)
names(adult)[51]="salary_bucket"
splitIndex <- createDataPartition(adult$salary_bucket, p = 0.70, list = FALSE, times = 1)
trainData <- adult[splitIndex,]
testData <- adult[-splitIndex,]
mytree <- rpart(salary_bucket ~ ., data = trainData, method = "class")
pred <- predict(mytree, testData, type = "class")
cm=confusionMatrix(data = (pred), reference = (testData$salary_bucket), positive = " >50K")
dataAccuracy<-cm[[3]][1]
dataBalancedAccuracy<-cm[[4]][11]
dataAccuracy
dataBalancedAccuracy

Created from 32561 samples and 51 variables

Pre-processing:
  - centered (50)
  - ignored (1)
  - 5 nearest neighbor imputation (50)
  - scaled (50)


Accuracy 
0.8443898

Balanced Accuracy 
        0.7407426

In [12]:
adult11 = read.csv("C:\\Users\\student\\Documents\\Statistical Analysis using R\\Assignment 10\\adult.csv", header = FALSE)
colnames(adult11) <- c("age", "workclass", "fnlwgt", "education", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "capital_gain", "capital_loss", "hours_per_week", "country", "salary_bucket")

levels(adult11$workclass)[levels(adult11$workingclass)==" ?"]<-NA
levels(adult11$occupation)[levels(adult11$occupation)==" ?"]<-NA
levels(adult11$country)[levels(adult11$country)==" ?"]<-NA

adult11<-replaceMissingValues(adult11)

levels(adult11$country)<-c("Asia","North_America","Asia","South_America","North_America","North_America","South_America","North_America","Europe","Europe","Europe","Europe","North_America","North_America","Europe","North_America","Asia","Europe","Asia","Asia","Europe","Europe","North_America","Asia","Asia","North_America","South_America","Oceanic","South_America","Oceanic","Europe","Europe","North_America","Europe","Asia","Asia","Asia","North_America","South_America","Asia","Europe")
levels(adult11$education)<-c("High_school","High_school","High_school","Elementary","Elementary","Elementary","High_school","Undergrad","Undergrad","Undergrad","Postgrad","High_school","Professiona","Elementary","Postgrad","Undergrad")
levels(adult11$occupation)<-c("Business","Government","Industry","Business","Agriculture","Business","Industry","Other","Business","Business","Government","Business","Business","Industry")

preProcessDataModel <- preProcess(adult11, method='knnImpute')
preProcessDataModel

if(!is.factor(adult11$salary_bucket)) {
  adult11$salary_bucket = factor(adult11$salary_bucket)
}
dummies_model <- dummyVars(salary_bucket~., data=adult11)


# Create the dummy variables using predict. The Y variable will not be present in trainData_mat.
trainData_mat <- predict(dummies_model, newdata = adult11)

# # Convert to dataframe
trainData <- data.frame(trainData_mat)
trainData$salary_bucket <- adult11$salary_bucket

adult11 = trainData

set.seed(2018)
names(adult11)[50]="salary_bucket"
splitIndex <- createDataPartition(adult11$salary_bucket, p = 0.70, list = FALSE, times = 1)
trainData <- adult11[splitIndex,]
testData <- adult11[-splitIndex,]
mytree <- rpart(salary_bucket ~ ., data = trainData, method = "class")
pred <- predict(mytree, testData, type = "class")
cm=confusionMatrix(data = (pred), reference = (testData$salary_bucket), positive = " >50K")
dataAccuracy<-cm[[3]][1]
dataBalancedAccuracy<-cm[[4]][11]
dataAccuracy
dataBalancedAccuracy

Created from 30162 samples and 15 variables

Pre-processing:
  - centered (6)
  - ignored (9)
  - 5 nearest neighbor imputation (6)
  - scaled (6)


Warning message in model.frame.default(Terms, newdata, na.action = na.action, xlev = object$lvls):
"variable 'salary_bucket' is not a factor"

Accuracy 
0.8443898

Balanced Accuracy 
        0.7407426

In [13]:
adult=read.csv("C:\\Users\\student\\Documents\\Statistical Analysis using R\\Assignment 10\\adult.csv", header = FALSE)
colnames(adult) <- c("age", "workclass", "fnlwgt", "education", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "capital_gain", "capital_loss", "hours_per_week", "country", "salary_bucket")

levels(adult$workclass)[levels(adult$workingclass)==" ?"]<-NA
levels(adult$occupation)[levels(adult$occupation)==" ?"]<-NA
levels(adult$country)[levels(adult$country)==" ?"]<-NA

adult<-replaceMissingValues(adult)

levels(adult$country)<-c("Asia","North_America","Asia","South_America","North_America","North_America","South_America","North_America","Europe","Europe","Europe","Europe","North_America","North_America","Europe","North_America","Asia","Europe","Asia","Asia","Europe","Europe","North_America","Asia","Asia","North_America","South_America","Oceanic","South_America","Oceanic","Europe","Europe","North_America","Europe","Asia","Asia","Asia","North_America","South_America","Asia","Europe")
levels(adult$education)<-c("High_school","High_school","High_school","Elementary","Elementary","Elementary","High_school","Undergrad","Undergrad","Undergrad","Postgrad","High_school","Professiona","Elementary","Postgrad","Undergrad")
levels(adult$occupation)<-c("Business","Government","Industry","Business","Agriculture","Business","Industry","Other","Business","Business","Government","Business","Business","Industry")

for(j in ncol(adult)) {
  adult[,j]<-as.numeric(adult[,j])
}

if(!is.factor(adult$salary_bucket)) {
  adult$salary_bucket = factor(adult$salary_bucket)
}
dummies_model <- dummyVars(salary_bucket~., data=adult)


# Create the dummy variables using predict. The Y variable will not be present in trainData_mat.
trainData_mat <- predict(dummies_model, newdata = adult)

# # Convert to dataframe
trainData <- data.frame(trainData_mat)
trainData$salary_bucket <- adult$salary_bucket

adult = trainData

preProcessDataModel <- preProcess(adult, method=c("scale","center"))
preProcessDataModel

set.seed(2018)
names(adult)[50]="salary_bucket"
splitIndex <- createDataPartition(adult$salary_bucket, p = 0.70, list = FALSE, times = 1)
trainData <- adult[splitIndex,]
testData <- adult[-splitIndex,]
mytree <- rpart(salary_bucket ~ ., data = trainData, method = "class")
pred <- predict(mytree, testData, type = "class")
cm=confusionMatrix(data = (pred), reference = (testData$salary_bucket), positive = "1")
dataAccuracy<-cm[[3]][1]
dataBalancedAccuracy<-cm[[4]][11]
dataAccuracy
dataBalancedAccuracy

Warning message in model.frame.default(Terms, newdata, na.action = na.action, xlev = object$lvls):
"variable 'salary_bucket' is not a factor"

Created from 30162 samples and 50 variables

Pre-processing:
  - centered (49)
  - ignored (1)
  - scaled (49)


Accuracy 
0.8443898

Balanced Accuracy 
        0.7407426

In [14]:
adult=read.csv("C:\\Users\\student\\Documents\\Statistical Analysis using R\\Assignment 10\\adult.csv", header = FALSE)
colnames(adult) <- c("age", "workclass", "fnlwgt", "education", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "capital_gain", "capital_loss", "hours_per_week", "country", "salary_bucket")

levels(adult$workclass)[levels(adult$workingclass)==" ?"]<-NA
levels(adult$occupation)[levels(adult$occupation)==" ?"]<-NA
levels(adult$country)[levels(adult$country)==" ?"]<-NA

adult<-replaceMissingValues(adult)

for(j in ncol(adult)) {
  adult[,j]<-as.numeric(adult[,j])
}

if(!is.factor(adult$salary_bucket)) {
  adult$salary_bucket = factor(adult$salary_bucket)
}
dummies_model <- dummyVars(salary_bucket~., data=adult)


# Create the dummy variables using predict. The Y variable will not be present in trainData_mat.
trainData_mat <- predict(dummies_model, newdata = adult)

# # Convert to dataframe
trainData <- data.frame(trainData_mat)
trainData$salary_bucket <- adult$salary_bucket

adult = trainData

preProcessDataModel <- preProcess(adult, method=c("scale","center"))
preProcessDataModel

set.seed(2018)
names(adult)[50]="salary_bucket"
splitIndex <- createDataPartition(adult$salary_bucket, p = 0.70, list = FALSE, times = 1)
trainData <- adult[splitIndex,]
testData <- adult[-splitIndex,]
mytree <- rpart(salary_bucket ~ ., data = trainData, method = "class")
pred <- predict(mytree, testData, type = "class")
cm=confusionMatrix(data = (pred), reference = as.factor(testData$salary_bucket), positive = "1")
dataAccuracy<-cm[[3]][1]
dataBalancedAccuracy<-cm[[4]][11]
dataAccuracy
dataBalancedAccuracy

Warning message in model.frame.default(Terms, newdata, na.action = na.action, xlev = object$lvls):
"variable 'salary_bucket' is not a factor"

Created from 30162 samples and 106 variables

Pre-processing:
  - centered (105)
  - ignored (1)
  - scaled (105)


Accuracy 
0.999181

Balanced Accuracy 
        0.9991883